### Ingest results.json file

##### Read the json file using spark df reader

In [0]:
dbutils.widgets.text("p_data_source","")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date","2021-03-28")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType, FloatType, TimestampType
from pyspark.sql.functions import col, current_timestamp, lit, to_timestamp,concat

In [0]:
results_schema = StructType(fields=[StructField("resultId", IntegerType(), True),
                                     StructField("raceId", IntegerType(), True),
                                     StructField("driverId", IntegerType(), True),
                                     StructField("constructorId", IntegerType(), True),
                                     StructField("number", IntegerType(), True),
                                     StructField("grid", IntegerType(), True),
                                     StructField("position", IntegerType(), True),
                                     StructField("positionText", StringType(), True),
                                     StructField("positionOrder", IntegerType(), True),
                                     StructField("points", FloatType(), True),
                                     StructField("lap", IntegerType(), True),
                                     StructField("time", StringType(), True),
                                     StructField("milliseconds", IntegerType(), True),
                                     StructField("fastestLap", IntegerType(), True),
                                     StructField("rank", IntegerType(), True),
                                     StructField("fastestLapTime", StringType(), True),
                                     StructField("fastestLapSpeed", StringType(), True),
                                     StructField("statusId", IntegerType(), True)])

In [0]:
results_df = spark.read \
.option("header", "true") \
.schema(results_schema) \
.json(f"{raw_inc_folder_path}/{v_file_date}/results.json")

##### Select the columns

In [0]:
results_with_columns_df = results_df.withColumnRenamed("resultId", "result_id") \
                                     .withColumnRenamed("raceId", "race_id") \
                                     .withColumnRenamed("driverId", "driver_id") \
                                     .withColumnRenamed("constructorId", "constructor_id") \
                                     .withColumnRenamed("positionText", "position_text") \
                                     .withColumnRenamed("positionOrder", "position_order") \
                                     .withColumnRenamed("fastestLap", "fastest_lap") \
                                     .withColumnRenamed("fastestLapTime", "fastest_lap_time") \
                                     .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed") \
                                     .withColumn("ingestion_date", current_timestamp()) \
                                     .withColumn("data_source", lit(v_data_source))\
                                     .withColumn("file_date", lit(v_file_date))

In [0]:
results_final_df = results_with_columns_df.drop("statusId")
                                          

In [0]:
results_final_df = results_final_df.dropDuplicates(["race_id","driver_id"])

Write the result as parquet file

Method 1 Incremental Load

In [0]:
# for race_id_list in results_final_df.select("race_id").distinct().collect():
#     if (spark._jsparkSession.catalog().tableExists("f1_processed_incremental.results")):
#         spark.sql(f"ALTER TABLE f1_processed_incremental.results DROP IF EXISTS PARTITION (race_id = {race_id_list.race_id})")


In [0]:
#results_final_df.write.mode("overwrite").partitionBy("race_id").parquet(f"{processed_folder_path}/results")

In [0]:
# results_final_df.write.mode("append").partitionBy("race_id").format("parquet").saveAsTable("f1_processed_incremental.results")

Method 2

In [0]:
#overwrite_partition(results_final_df, "f1_processed_incremental", "results", "race_id")

In [0]:
merge_condition =  "tgt.result_id = src.result_id AND tgt.race_id = src.race_id"
merge_delta_data(results_final_df, "f1_processed_incremental", "results", processed_inc_folder_path, merge_condition, "race_id")

In [0]:
%sql
SELECT * FROM f1_processed_incremental.results

In [0]:
display(spark.read.format("delta").load(f"{processed_inc_folder_path}/results"))

In [0]:
%sql
SELECT race_id, count(1)
FROM f1_processed_incremental.results
group by race_id

In [0]:
dbutils.notebook.exit("Success")